# <a href="https://thetahat.ru/courses/ph-ds-2024-aut">Phystech@DataScience</a>
## Семинар 6

**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* **Код из рассказанных на занятиях ноутбуков можно использовать без ограничений.**

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

In [22]:
# Bot check

# HW_ID: phds_sem6
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [23]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance

import seaborn as sns

sns.set(context='poster')
%matplotlib inline

### Задача 1

Рассмотрим как можно провести отбор признаков с помощью обучения линейной регрессии и леса с малым количеством деревьев ($\approx 10$) на примере задачи регрессии. Будем использовать датасет <https://www.kaggle.com/datasets/abrambeyer/openintro-possum> и пытаться предсказать возраст оппосумов на основе различных параметров их тела.

Для упрощения задачи избавимся от категориальных признаков

In [24]:
data = pd.read_csv('possum.csv').dropna()
X = data[data.columns.drop(['sex', 'Pop', 'age'])]
y = data['age']

Разделим выборку на тренировочную и тестовую часть

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные для корректной работы **линейной регрессии**:

In [26]:
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

Обучим модель линейной регрессии с L1-регуляризацией на всех данных, посмотрим на метрику качества

In [27]:
lin_model = Lasso(alpha=0.1)
lin_model.fit(X_train_norm, y_train)
print('Коэффиценты модели:', lin_model.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model.predict(X_train_norm), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model.predict(X_test_norm), y_test))

Коэффиценты модели: [-0.          0.          0.2451365   0.0463782   0.         -0.
 -0.07870314  0.          0.23833098  0.05802872  0.30598817]
Значение MSE на тренировочной выборке: 3.0907392585723974
Значение MSE на тестовой выборке: 2.955306344756532


Проделаем аналогичную процедуру с лесом c небольшим количеством деревьев:

In [28]:
forest = RandomForestRegressor(n_estimators=10, random_state=42)
forest.fit(X_train, y_train)
print('Важность признаков:', forest.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest.predict(X_train), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest.predict(X_test), y_test))

Важность признаков: [0.13901666 0.00976327 0.21418161 0.28668792 0.0726663  0.05745777
 0.03598931 0.07092169 0.04461522 0.05049527 0.01820497]
Значение MSE на тренировочной выборке: 0.5725373134328358
Значение MSE на тестовой выборке: 2.6567647058823534


Что вы можете сказать смотря на коэффициенты модели для регрессии и на важность признаков для леса?

**Вывод**: В модели линейной регрессии есть нулевые коэффициенты, это о говорит о том, что эти признаки не влияют, это специфика lasso регрессии. Важность/коэффициенты можно отранжировать, и отобрать самые важные признаки.

Составим 2 списка наиболее важных признаков - один с признаками, наиболее важными для линейной регрессии, второй с наиболее важными для леса

In [29]:
lin_imp = []
forest_imp = []
for i, column in enumerate(X.columns):
    if lin_model.coef_[i] > 0.1:
        lin_imp.append(i)
    if forest.feature_importances_[i] > 0.1:
        forest_imp.append(column)

In [30]:
X_train_norm_lin_imp = X_train_norm[:, lin_imp]
X_test_norm_lin_imp = X_test_norm[:, lin_imp]

X_train_forest_imp = X_train[forest_imp]
X_test_forest_imp = X_test[forest_imp]

Теперь обучим модели только на важных признаках:

In [31]:
lin_model_2 = Lasso(alpha=0.1)
lin_model_2.fit(X_train_norm_lin_imp, y_train)
print('Коэффиценты модели:', lin_model_2.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model_2.predict(X_train_norm_lin_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model_2.predict(X_test_norm_lin_imp), y_test))

Коэффиценты модели: [0.27942877 0.23624623 0.30994129]
Значение MSE на тренировочной выборке: 3.1274779658965923
Значение MSE на тестовой выборке: 2.9453030952790327


In [32]:
forest_2 = RandomForestRegressor(n_estimators=10, random_state=42)
forest_2.fit(X_train_forest_imp, y_train)
print('Важность признаков:', forest_2.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest_2.predict(X_train_forest_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest_2.predict(X_test_forest_imp), y_test))

Важность признаков: [0.24975552 0.3691183  0.38112618]
Значение MSE на тренировочной выборке: 0.5901492537313433
Значение MSE на тестовой выборке: 2.3205882352941174


Что вы можете сказать о качестве предсказания?

**Вывод:** mse осталось примерно таким же, где-то даже увеличилось, это говорит о потере важной информации из-за исключения некоторых признаков. В случае леса это может говорить о потере нелинейный взимодействий после удаления признаков. У нас изначально было не очень много признаков, соответственно, можно было и не проводить отбор.

Рассмотрим работу других метотодов оценки важности признаков, а именно *Permutation feature importance* и *Column feature importance*,на примере [KNN-регресии](https://scikit-learn.org/1.5/modules/generated/sklearn.neighbors.KNeighborsRegressor.html).

Для корректной работы KNN необходимо стандартизовать признаки, как мы это делали для Lasso-регресии. А также разделить тренировачный датасет на train и val.

In [33]:
X_train_norm, X_val_norm, y_train, y_val = train_test_split(X_train_norm, y_train, test_size=0.33, random_state=42)

In [34]:
knn_1 = KNeighborsRegressor()
knn_1.fit(X_train_norm, y_train)
print('Значение MSE на тренировочной выборке:', MSE(knn_1.predict(X_train_norm), y_train))
print('Значение MSE на тестовой выборке:', MSE(knn_1.predict(X_test_norm), y_test))

Значение MSE на тренировочной выборке: 2.5009090909090914
Значение MSE на тестовой выборке: 2.449411764705882


**Permutation feature importance** полностью реализован в `sklearn.inspection`


Функция `permutation_importance()` принимает на вход:
- `model` &mdash; обученная модель
- `X, y` &mdash;  фичи и таргет валидационной части датасета
- `n_repeats` &mdash; сколько раз переставляется фича

На выходе мы получаем:
- `importances` &mdash сырые оценки значимости для всех фичей и всех итераций
- `importances_mean` &mdash; среднее по всем итерациям
- `importances_std` &mdash; стандартоное отклонение среднего

Оценим важность признаков

In [35]:
r = permutation_importance(knn_1, X_val_norm, y_val, n_repeats=30, random_state=42)

Отберем признаки согласно нашей оценке

In [36]:
knn_imp = []
for i, column in enumerate(X.columns):
    if r.importances_mean[i] - r.importances_std[i] >= 0:
        knn_imp.append(i)

In [42]:
X_train_norm_knn_imp = X_train_norm[:, knn_imp]
X_test_norm_knn_imp = X_test_norm[:, knn_imp]

In [41]:
knn_2 = KNeighborsRegressor(n_neighbors=5)
knn_2.fit(X_train_norm_knn_imp, y_train)
print('Значение MSE на тренировочной выборке:', MSE(knn_2.predict(X_train_norm_knn_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(knn_2.predict(X_test_norm_knn_imp), y_test))

Значение MSE на тренировочной выборке: 3.277272727272728
Значение MSE на тестовой выборке: 3.1341176470588232


**Drop-Column feature importance**

Для него готовой реализации в `sklearn` нет, так что воспольлзуемся кодом ниже.

In [45]:
knn = KNeighborsRegressor(n_neighbors=5).fit(X_train_norm, y_train)
baseline = MSE(knn.predict(X_val_norm), y_val)
knn_imp = []

for i, column in enumerate(X.columns):
    X_train_drop = np.delete(X_train_norm, i, 1)
    X_val_drop =  np.delete(X_val_norm, i , 1)

    knn_drop =KNeighborsRegressor(n_neighbors=5)
    knn_drop.fit(X_train_drop, y_train)
    mse = MSE(knn_drop.predict(X_val_drop), y_val)

    if ((mse - baseline) / baseline) > 0.1:
        knn_imp.append(i)

In [46]:
X_train_norm_knn_imp = X_train_norm[:, knn_imp]
X_test_norm_knn_imp = X_test_norm[:, knn_imp]

In [48]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_norm_knn_imp, y_train)
print('Значение MSE на тренировочной выборке:', MSE(knn.predict(X_train_norm_knn_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(knn.predict(X_test_norm_knn_imp), y_test))

Значение MSE на тренировочной выборке: 2.4309090909090916
Значение MSE на тестовой выборке: 2.4870588235294115


**Вывод** knn основывается на ближайших соседях при предсказании, если мы переставляем признаки (Permutation feature importance),это может ухудшить сильно качество (у нас так и вышло, mse выросло).

Во втором случае (Drop-Column feature importance) мы поочередно удаляем признаки, отбираем самые важные, но mse осталось примерно таким же (по сравнению с изначальными данными), вероятно, мы удалили довольно важные признаки, которые влияли на предсказание.